# **Learning**

In [7]:
import pandas as pd
import numpy as np

from tensorflow import keras
import tensorflow as tf

## **Loading training dataset**

In [8]:
TRAINING_SET_FEATURES = "../01_data/train/under_educated_features.csv"
TRAINING_SET_LABELS = "../01_data/train/under_educated_labels.csv"

In [9]:
X = np.loadtxt(TRAINING_SET_FEATURES)
y = np.loadtxt(TRAINING_SET_LABELS)

In [10]:
X.shape, y.shape

((20326, 90), (20326,))

## **Training a neural network**

### **Definition of the network**

In [14]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(X.shape[1])),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="sigmoid"),
    keras.layers.Dense(200, activation="sigmoid"),
    keras.layers.Dense(500, activation="sigmoid"),
    keras.layers.Dense(200, activation="sigmoid"),
    keras.layers.Dense(100, activation="sigmoid"),
    keras.layers.Dense(1)
])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [16]:
opt = keras.optimizers.Adam(learning_rate=0.05)
model.compile(optimizer=opt, loss="mse")
model.fit(X, y, validation_split=0.75, epochs=10, batch_size=500)

Train on 5081 samples, validate on 15245 samples
Epoch 1/10
5081/5081 [==============================] - 1s 124us/sample - loss: 12.6087 - val_loss: 4.3862
Epoch 2/10
5081/5081 [==============================] - 0s 54us/sample - loss: 2.3104 - val_loss: 2.0387
Epoch 3/10
5081/5081 [==============================] - 0s 57us/sample - loss: 1.5101 - val_loss: 0.9973
Epoch 4/10
5081/5081 [==============================] - 0s 54us/sample - loss: 1.2214 - val_loss: 1.0406
Epoch 5/10
5081/5081 [==============================] - 0s 56us/sample - loss: 1.1283 - val_loss: 0.9712
Epoch 6/10
5081/5081 [==============================] - 0s 52us/sample - loss: 1.0866 - val_loss: 0.9954
Epoch 7/10
5081/5081 [==============================] - 0s 57us/sample - loss: 1.0844 - val_loss: 0.9766
Epoch 8/10
5081/5081 [==============================] - 0s 52us/sample - loss: 1.0830 - val_loss: 0.9882
Epoch 9/10
5081/5081 [==============================] - 0s 57us/sample - loss: 1.0839 - val_loss: 1.0016
Epoc